In [ ]:
from __future__ import print_function

import numpy as np
import glob


## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
        

from clustered_SNe.analysis.constants import m_proton, pc, yr, M_solar, \
                                   metallicity_solar
from clustered_SNe.analysis.parse import Overview, RunSummary, \
                                         Inputs, \
                                         get_full_id_from_partial_id, \
                                         checkpoint_num_from_filename

from clustered_SNe.analysis.database_helpers import session, \
                                                Simulation, \
                                                Simulation_Inputs, \
                                                Simulation_Status


In [ ]:
data_dir = "../saved_runs/cluster_parameter_study/"

In [ ]:
ids = [
"32226257-da2d-47ae-85ef-d99b39621202",
"20b4fccc-e53f-48ca-894b-3ee282bad8ea",
"3cf9d3f4-b657-4437-a59f-324de0b52763",
"db48961d-c5d2-4495-8c47-a1922d0170a9",
"45a6c151-774b-479f-b4fc-83c9945516c7",
"b7ef7e0d-72b5-4592-ad9a-288fa98fd7cc",
"af79be27-ff8e-47c1-bee1-e29a2786d908",
"e219741b-5ff9-488c-8a0b-c64b6f568c00",
"9504c81d-b2f7-4593-8c28-e661c501cb11",
"c436d5db-4feb-41a0-8175-f6dd2287a6b9",
"e3d7135a-0265-4216-b63f-e29ba039f912",
"c7b202a2-dbf2-46fb-a192-4ab258b70b9d"
]

# ids = [get_full_id_from_partial_id(data_dir, partial_id) for partial_id in ids]


In [ ]:
qsub_systems = set(["SGE", "PBS"])
qsub_system = "SGE"
assert(qsub_system in qsub_systems)
    
f_submit = open("submit_all", mode="w")
f_submit.write("#!/bin/bash \n")

for id in ids:

    print("id: ", id)
    
    overview = Overview(os.path.join(data_dir, id + "_overview.dat"))
    
    filenames = glob.glob(os.path.join(data_dir, id + "_checkpoint_*.dat"))
    filenames = sorted(filenames)
    last_checkpoint_filename  = filename[-1]
    last_checkpoint_number = checkpoint_num_from_filename(last_checkpoint_filename)
    
    with open(last_checkpoint_filename, 'r') as f:
        line = f.readline()
        last_checkpoint_time = float(line.split()[3])
    
    restart_checkpoints_needed = 100 - (last_checkpoint_number+1 % 100)
    print("new checkpoints_needed: ", restart_checkpoints_needed)
    
    restart_time_needed = overview.SNe_times.max() * 3 - last_checkpoint_times
    if last_checkpoint_number >= 99:
        restart_time_needed = 3e15
    print("extra time needed: ", restart_time_needed, "[s]")

    print("")

    CFL = .025

    f_submit.write("qsub " +
                       "-v RESTART_ID=" + run_summary.id + ","
                       + "RESTART_N_CHECKPOINTS=" + str(restart_checkpoints_needed) + ","
                       + "RESTART_DELTA_TIME=" + str(restart_time_needed) + ","
                       + "RESTART_CFL=" + str(CFL)
                    + "../scripts/" + qsub_system + "/restart.batch.env_variables \n")
        
f_submit.close()

In [ ]:
for id in ids:
#     full_id = get_full_id_from_partial_id(data_dir, id)
    session.query(Simulation_Status).filter(Simulation_Status.id == id).update({"status":"Running"})
    sim_status = session.query(Simulation_Status).get(id)
    print(sim_status.status)
    print(id)

In [ ]:
session.commit()